In [16]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime
import dateutil.parser
#from uszipcode import SearchEngine
import warnings
warnings.filterwarnings("ignore")

In [17]:
# retrieve data for year 2017
#url_2017 = "https://data.cityofchicago.org/resource/crimes.json?$where=year=2017&$limit=50000"
#response_2017 = requests.get(url_2017).json()
#df_2017 = pd.DataFrame(response_2017)

In [18]:
# retrieve data for year 2018
url_2018 = "https://data.cityofchicago.org/resource/crimes.json?$where=year=2018&$limit=50000"
response_2018 = requests.get(url_2018).json()
df_2018 = pd.DataFrame(response_2018)

In [19]:
# retrieve data for year 2019
url_2019 = "https://data.cityofchicago.org/resource/crimes.json?$where=year=2019&$limit=50000"
response_2019 = requests.get(url_2019).json()
df_2019 = pd.DataFrame(response_2019)


In [20]:
# retrieve data for year 2020
url_2020 = "https://data.cityofchicago.org/resource/crimes.json?$where=year=2020&$limit=50000"
response_2020 = requests.get(url_2020).json()
df_2020 = pd.DataFrame(response_2020)

In [21]:
# retrieve data for year 2021
url_2021 = "https://data.cityofchicago.org/resource/crimes.json?$where=year=2021&$limit=50000"
response_2021 = requests.get(url_2021).json()
df_2021 = pd.DataFrame(response_2021)

In [22]:
# concatenate all the dataframes
df = pd.concat([df_2018, df_2019, df_2020, df_2021])
df=df.reset_index(drop=True)
df.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,year,updated_on,x_coordinate,y_coordinate,latitude,longitude,location
0,11646166,JC213529,2018-09-01T00:01:00.000,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8,44,06,2018,2019-04-06T16:04:43.000,NaN,NaN,NaN,NaN,NaN
1,11645648,JC212959,2018-01-01T08:00:00.000,024XX N MONITOR AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,30,19,11,2018,2019-04-06T16:04:43.000,NaN,NaN,NaN,NaN,NaN
2,11645959,JC211511,2018-12-20T16:00:00.000,045XX N ALBANY AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,...,33,14,08A,2018,2019-04-06T16:04:43.000,NaN,NaN,NaN,NaN,NaN
3,11645557,JC212685,2018-04-01T00:01:00.000,080XX S VERNON AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,6,44,11,2018,2019-04-06T16:04:43.000,NaN,NaN,NaN,NaN,NaN
4,11646293,JC213749,2018-12-20T15:00:00.000,023XX N LOCKWOOD AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,False,False,...,36,19,11,2018,2019-04-06T16:04:43.000,NaN,NaN,NaN,NaN,NaN


In [130]:
df.columns

Index(['id', 'case_number', 'date', 'block', 'iucr', 'primary_type',
       'description', 'location_description', 'arrest', 'domestic', 'beat',
       'district', 'ward', 'community_area', 'fbi_code', 'year', 'updated_on',
       'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'location'],
      dtype='object')

In [136]:
df['longitude']=pd.to_numeric(df['longitude'])
df['latitude']=pd.to_numeric(df['latitude'])

In [174]:
# import numpy as np
import geopy
# import pandas as pd

sample=df.tail()
sample

# longlat = pd.read_csv("longlat.csv",encoding = 'utf-8', sep = '\t')
geolocator = geopy.Nominatim(user_agent="check_1")

def get_zip_code(x):
    # if x[['']]
    if (np.isnan(x['latitude'])) & (np.isnan(x['longitude'])): 
        return None
    else: 
        location = geolocator.reverse("{}, {}".format(x['latitude'],x['longitude']))
        return location.raw['address']['postcode']

zip_df=sample.apply(lambda x: get_zip_code(x), axis = 1)
# longlat['zipcode'] = longlat.head().apply(lambda x: get_zip_code(x), axis = 1)
# print(longlat.head())
zip_df

49995    60645
49996     None
49997    60634
49998    60623
49999    60611
dtype: object

In [202]:
for i, (idx, row) in enumerate(df.iterrows()): 
    if (i % 100) == 0: 
        print(f'Working on {i}th record. ')
    # df.loc[idx, 'zip']=row.map(lambda x: get_zip_code(x))
    df.loc[idx, 'zip']=get_zip_code(row)

Working on 0th record. 
Working on 100th record. 
Working on 200th record. 
Working on 300th record. 
Working on 400th record. 


KeyboardInterrupt: 

In [201]:
# df['zip']=df.apply(lambda x: get_zip_code(x), axis = 1)
# df.head()

KeyboardInterrupt: 

In [203]:
pivot_df=df.pivot_table(index='primary_type', columns='year', aggfunc='size')
updated_pivot_df=pivot_df.div(pivot_df.sum(axis=0), axis=1)
updated_pivot_df

year,2018,2019,2020,2021
primary_type,,,,
ARSON,0.00126,0.00140,0.00224,0.00218
ASSAULT,0.07594,0.07774,0.08960,0.09816
BATTERY,0.18706,0.18914,0.20956,0.19678
BURGLARY,0.04228,0.03878,0.04384,0.03106
CONCEALED CARRY LICENSE VIOLATION,0.00032,0.00064,0.00052,0.00094
CRIM SEXUAL ASSAULT,0.00512,0.00388,0.00020,NaN
CRIMINAL DAMAGE,0.10238,0.10006,0.12738,0.12242
CRIMINAL SEXUAL ASSAULT,0.00006,0.00058,0.00454,0.00614
CRIMINAL TRESPASS,0.03018,0.02836,0.01864,0.01656


In [205]:
pivot_df=df.pivot_table(index='zip', columns='year', aggfunc='size')
# updated_pivot_df=pivot_df.div(pivot_df.sum(axis=0), axis=1)
# updated_pivot_df
pivot_df

year,2018,2019,2020,2021
zip,,,,
60302,1,1,1,1
60409,1,1,1,1
60605,2,2,2,2
60606,1,1,1,1
60607,2,2,2,2
60609,2,2,2,2
60610,2,2,2,2
60611,1,1,1,1
60612,2,2,2,2


In [ ]:
df['2019_percent_change']=df['2019']-df['2018']

In [ ]:
# scatter plot of changes vs. economic changes

In [ ]:
# analysis 
    # types of crime change over time
    # most popular types of crime at well known zip codes, i.e. where homicide happens often
    # amount of crime per zip code change over time
    # layer in economic change per zip code
    # when do certain times of crime occur in the year
    
# slide show
# README

In [127]:
df[['year', 'primary_type']].value_counts()

year  primary_type            
2019  THEFT                       11796
2018  THEFT                       11402
2020  BATTERY                     10478
2021  BATTERY                      9839
2020  THEFT                        9796
                                  ...  
2021  HUMAN TRAFFICKING               2
      NON-CRIMINAL                    1
2020  HUMAN TRAFFICKING               1
      OTHER NARCOTIC VIOLATION        1
2019  NON-CRIMINAL                    1
Length: 121, dtype: int64

In [ ]:
df.to_csv('updated_data.csv', index=False)

# Final DataFrame for Chicago from 2018-2021

In [94]:
columns = ["year","latitude","longitude","location"]
chicago_df = df[columns]
chicago_df

,year,latitude,longitude,location
0,2018,NaN,NaN,NaN
1,2018,NaN,NaN,NaN
2,2018,NaN,NaN,NaN
3,2018,NaN,NaN,NaN
4,2018,NaN,NaN,NaN
...,...,...,...,...
49995,2021,42.005604761,-87.681877001,"{'latitude': '42.005604761', 'longitude': '-87..."
49996,2021,NaN,NaN,NaN
49997,2021,41.942130194,-87.774881657,"{'latitude': '41.942130194', 'longitude': '-87..."
49998,2021,41.851761023,-87.705558672,"{'latitude': '41.851761023', 'longitude': '-87..."


# la data

In [52]:
# the 2017
#url = "https://data.lacity.org/resource/63jg-8b9z.json?$limit=50000&$where=date_rptd%20between%20%272017-01-01T00:00:00%27%20and%20%272017-12-31T23:59:59%27"
#response = requests.get(url)
#data = response.json()
#la_2017_df = pd.DataFrame(data)
#la_2017_df

In [53]:
# the 2018 
url = "https://data.lacity.org/resource/63jg-8b9z.json?$limit=50000&$where=date_rptd%20between%20%272018-01-01T00:00:00%27%20and%20%272018-12-31T23:59:59%27"
response = requests.get(url)
data = response.json()
la_2018_df = pd.DataFrame(data)


In [54]:
# the 2019 
url = "https://data.lacity.org/resource/63jg-8b9z.json?$limit=50000&$where=date_rptd%20between%20%272019-01-01T00:00:00%27%20and%20%272019-12-31T23:59:59%27"
response = requests.get(url)
data = response.json()
la_2019_df = pd.DataFrame(data)


In [55]:
# Crime Data from 2020 

url_2020= "https://data.lacity.org/resource/63jg-8b9z.json?$limit=638245&$where=date_rptd%20between%20%272020-01-01T00:00:00%27%20and%20%272020-12-31T23:59:59%27"


response_2020 = requests.get(url_2020).json()
la_2020_df=pd.DataFrame(response_2020)


In [56]:
# retrieve data for year 2021
url_2021= "https://data.lacity.org/resource/63jg-8b9z.json?$limit=638245&$where=date_rptd%20between%20%272021-01-01T00:00:00%27%20and%20%272021-12-31T23:59:59%27"


response_2021 = requests.get(url_2021).json()
la_2021_df=pd.DataFrame(response_2021)

In [18]:
#url_2022= "https://data.lacity.org/resource/63jg-8b9z.json?$limit=638245&$where=date_rptd%20between%20%272022-01-01T00:00:00%27%20and%20%272022-12-31T23:59:59%27"


#response_2022 = requests.get(url_2022).json()
#la_2022_df=pd.DataFrame(response_2022)
#la_2022_df

In [19]:
#url_2023= "https://data.lacity.org/resource/63jg-8b9z.json?$limit=638245&$where=date_rptd%20between%20%272023-01-01T00:00:00%27%20and%20%272023-01-31T23:59:59%27"


#response_2023 = requests.get(url_2023).json()
#la_2023_df=pd.DataFrame(response_2023)
#la_2023_df

In [57]:
# concatenate all the dataframes
df1 = pd.concat([la_2018_df, la_2019_df,la_2020_df, la_2021_df])

df1

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,status,status_desc,crm_cd_1,location,lat,lon,cross_street,crm_cd_2,crm_cd_3,crm_cd_4
0,181104053,2018-01-01T00:00:00.000,2012-01-01T00:00:00.000,0001,11,Northeast,1177,1,821,SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO...,...,AO,Adult Other,821,2600 MERCED ST,34.0898,-118.2268,NaN,NaN,NaN,NaN
1,182104017,2018-01-01T00:00:00.000,2016-12-01T00:00:00.000,1200,21,Topanga,2196,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,4600 CANOGA AV,34.1516,-118.5955,NaN,NaN,NaN,NaN
2,181404045,2018-01-01T00:00:00.000,2017-12-21T00:00:00.000,0800,14,Pacific,1469,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,IC,Invest Cont,440,5700 W CENTINELA AV,33.9765,-118.381,NaN,NaN,NaN,NaN
3,180904025,2018-01-01T00:00:00.000,2017-12-30T00:00:00.000,1500,09,Van Nuys,0906,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740,14400 SATICOY ST,34.2085,-118.448,NaN,NaN,NaN,NaN
4,180104015,2018-01-01T00:00:00.000,2017-12-29T00:00:00.000,1100,01,Central,0166,1,210,ROBBERY,...,IC,Invest Cont,210,6TH ST,34.0423,-118.2452,SAN PEDRO,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,220204007,2021-12-31T00:00:00.000,2015-06-25T00:00:00.000,1200,02,Rampart,0269,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,700 S BIXEL ST,34.0509,-118.2642,NaN,NaN,NaN,NaN
2690,220704471,2021-12-31T00:00:00.000,2016-03-29T00:00:00.000,0940,07,Wilshire,0775,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354,4800 SATURN ST,34.044,-118.3415,NaN,NaN,NaN,NaN
2691,220706245,2021-12-31T00:00:00.000,2017-04-01T00:00:00.000,1500,07,Wilshire,0721,2,860,BATTERY WITH SEXUAL CONTACT,...,IC,Invest Cont,860,8700 W BEVERLY BL,34.0768,-118.3806,NaN,NaN,NaN,NaN
2692,211423712,2021-12-31T00:00:00.000,2019-03-01T00:00:00.000,1700,14,Pacific,1476,2,940,EXTORTION,...,IC,Invest Cont,940,6500 W 84TH ST,33.9629,-118.4019,NaN,NaN,NaN,NaN


In [58]:
df1.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1_2', 'crm_cd', 'crm_cd_desc', 'mocodes',
       'vict_age', 'vict_sex', 'vict_descent', 'premis_cd', 'premis_desc',
       'weapon_used_cd', 'weapon_desc', 'status', 'status_desc', 'crm_cd_1',
       'location', 'lat', 'lon', 'cross_street', 'crm_cd_2', 'crm_cd_3',
       'crm_cd_4'],
      dtype='object')

In [95]:
columns1 = ["date_rptd","location","lat","lon"]
la_df = df1[columns1]
la_df

,date_rptd,location,lat,lon
0,2018-01-01T00:00:00.000,2600 MERCED ST,34.0898,-118.2268
1,2018-01-01T00:00:00.000,4600 CANOGA AV,34.1516,-118.5955
2,2018-01-01T00:00:00.000,5700 W CENTINELA AV,33.9765,-118.381
3,2018-01-01T00:00:00.000,14400 SATICOY ST,34.2085,-118.448
4,2018-01-01T00:00:00.000,6TH ST,34.0423,-118.2452
...,...,...,...,...
2689,2021-12-31T00:00:00.000,700 S BIXEL ST,34.0509,-118.2642
2690,2021-12-31T00:00:00.000,4800 SATURN ST,34.044,-118.3415
2691,2021-12-31T00:00:00.000,8700 W BEVERLY BL,34.0768,-118.3806
2692,2021-12-31T00:00:00.000,6500 W 84TH ST,33.9629,-118.4019


In [96]:
la_df['Year'] = pd.DatetimeIndex(la_df['date_rptd']).year
la_df   

,date_rptd,location,lat,lon,Year
0,2018-01-01T00:00:00.000,2600 MERCED ST,34.0898,-118.2268,2018
1,2018-01-01T00:00:00.000,4600 CANOGA AV,34.1516,-118.5955,2018
2,2018-01-01T00:00:00.000,5700 W CENTINELA AV,33.9765,-118.381,2018
3,2018-01-01T00:00:00.000,14400 SATICOY ST,34.2085,-118.448,2018
4,2018-01-01T00:00:00.000,6TH ST,34.0423,-118.2452,2018
...,...,...,...,...,...
2689,2021-12-31T00:00:00.000,700 S BIXEL ST,34.0509,-118.2642,2021
2690,2021-12-31T00:00:00.000,4800 SATURN ST,34.044,-118.3415,2021
2691,2021-12-31T00:00:00.000,8700 W BEVERLY BL,34.0768,-118.3806,2021
2692,2021-12-31T00:00:00.000,6500 W 84TH ST,33.9629,-118.4019,2021


#  final DataFrame for LA from 2018 to 2021

In [97]:
columns1 = ["year","latitude","longitude","location"]
la_df = df[columns1]
la_df

,year,latitude,longitude,location
0,2018,NaN,NaN,NaN
1,2018,NaN,NaN,NaN
2,2018,NaN,NaN,NaN
3,2018,NaN,NaN,NaN
4,2018,NaN,NaN,NaN
...,...,...,...,...
49995,2021,42.005604761,-87.681877001,"{'latitude': '42.005604761', 'longitude': '-87..."
49996,2021,NaN,NaN,NaN
49997,2021,41.942130194,-87.774881657,"{'latitude': '41.942130194', 'longitude': '-87..."
49998,2021,41.851761023,-87.705558672,"{'latitude': '41.851761023', 'longitude': '-87..."


In [108]:
ny_url = "https://data.cityofnewyork.us/resource/8h9b-rp9u.json?$where=arrest_date between '2018-01-01T00:00:00' and '2021-12-31T23:59:59'&$limit=50000"
ny_response= requests.get(ny_url).json()
df3=pd.DataFrame(ny_response)
df3.head()

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat
0,173151555,2018-01-01T00:00:00.000,109,"ASSAULT 2,1,UNCLASSIFIED",106,FELONY ASSAULT,PL 1201001,F,K,66,0,25-44,M,ASIAN / PACIFIC ISLANDER,986735,167242,40.62572501600005,-73.99104937999994,"{'type': 'Point', 'coordinates': [-73.99104937..."
1,173148126,2018-01-01T00:00:00.000,531,"DRUG PARAPHERNALIA, POSSESSES OR SELLS 1",117,DANGEROUS DRUGS,PL 2205500,F,M,17,97,25-44,F,BLACK,992029,213332,40.752228400000035,-73.97192485799998,"{'type': 'Point', 'coordinates': [-73.97192485..."
2,173142262,2018-01-01T00:00:00.000,113,"MENACING,UNCLASSIFIED",344,ASSAULT 3 & RELATED OFFENSES,PL 1201401,M,S,121,0,<18,F,BLACK,939367,169549,40.63194388100004,-74.16170488099993,"{'type': 'Point', 'coordinates': [-74.16170488..."
3,173142129,2018-01-01T00:00:00.000,904,"INTOXICATED DRIVING,ALCOHOL",119,INTOXICATED/IMPAIRED DRIVING,VTL11920D3,F,B,40,2,45-64,M,BLACK HISPANIC,1006777,236405,40.81553235500007,-73.91861784599996,"{'type': 'Point', 'coordinates': [-73.91861784..."
4,173142152,2018-01-01T00:00:00.000,101,ASSAULT 3,344,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,B,46,0,18-24,M,BLACK HISPANIC,1008861,247665,40.84643225400004,-73.91104764699998,"{'type': 'Point', 'coordinates': [-73.91104764..."


In [122]:
df3.columns

Index(['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc',
       'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct',
       'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd',
       'y_coord_cd', 'latitude', 'longitude', 'lon_lat'],
      dtype='object')

In [123]:
ny_df['year'] = pd.DatetimeIndex(ny_df['arrest_date']).year
ny_df 

,arrest_date,latitude,longitude,lon_lat,year
0,2018-01-01T00:00:00.000,40.62572501600005,-73.99104937999994,"{'type': 'Point', 'coordinates': [-73.99104937...",2018
1,2018-01-01T00:00:00.000,40.752228400000035,-73.97192485799998,"{'type': 'Point', 'coordinates': [-73.97192485...",2018
2,2018-01-01T00:00:00.000,40.63194388100004,-74.16170488099993,"{'type': 'Point', 'coordinates': [-74.16170488...",2018
3,2018-01-01T00:00:00.000,40.81553235500007,-73.91861784599996,"{'type': 'Point', 'coordinates': [-73.91861784...",2018
4,2018-01-01T00:00:00.000,40.84643225400004,-73.91104764699998,"{'type': 'Point', 'coordinates': [-73.91104764...",2018
...,...,...,...,...,...
49995,2018-03-07T00:00:00.000,40.623226819000074,-74.14922697299994,"{'type': 'Point', 'coordinates': [-74.14922697...",2018
49996,2018-03-07T00:00:00.000,40.70148881400007,-74.01254496799999,"{'type': 'Point', 'coordinates': [-74.01254496...",2018
49997,2018-03-07T00:00:00.000,40.83564782100007,-73.88561001399995,"{'type': 'Point', 'coordinates': [-73.88561001...",2018
49998,2018-03-07T00:00:00.000,40.79863330000006,-73.94162037099994,"{'type': 'Point', 'coordinates': [-73.94162037...",2018


# final DataFrame for Nyc from 2018 to 2021

In [124]:
columns2 = ["year","latitude","longitude","lon_lat"]
nyc_df = ny_df[columns2]
nyc_df 

,year,latitude,longitude,lon_lat
0,2018,40.62572501600005,-73.99104937999994,"{'type': 'Point', 'coordinates': [-73.99104937..."
1,2018,40.752228400000035,-73.97192485799998,"{'type': 'Point', 'coordinates': [-73.97192485..."
2,2018,40.63194388100004,-74.16170488099993,"{'type': 'Point', 'coordinates': [-74.16170488..."
3,2018,40.81553235500007,-73.91861784599996,"{'type': 'Point', 'coordinates': [-73.91861784..."
4,2018,40.84643225400004,-73.91104764699998,"{'type': 'Point', 'coordinates': [-73.91104764..."
...,...,...,...,...
49995,2018,40.623226819000074,-74.14922697299994,"{'type': 'Point', 'coordinates': [-74.14922697..."
49996,2018,40.70148881400007,-74.01254496799999,"{'type': 'Point', 'coordinates': [-74.01254496..."
49997,2018,40.83564782100007,-73.88561001399995,"{'type': 'Point', 'coordinates': [-73.88561001..."
49998,2018,40.79863330000006,-73.94162037099994,"{'type': 'Point', 'coordinates': [-73.94162037..."


In [ ]:
# concatenate all the dataframes
final_df = pd.concat([df_2018, df_2019, df_2020, df_2021])

df.head()